In [1]:
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset # Network Common Data Form

In [2]:
def AlignDatasets(time, lat, lon, precipDayXX, rootgrp_obs, userInputs):
    
    dayOfTheSeason = userInputs[0]  
    lats           = userInputs[1]
    latn           = userInputs[2]
    lonw           = userInputs[3]
    lone           = userInputs[4]
    
    time_obs_raw = rootgrp_obs.variables['time'][:]   # "Days since 1900-01-01" The number of days till Sept 1, 2009 is 40,085
    lat_obs      = rootgrp_obs.variables['lat'][:]
    lon_obs      = rootgrp_obs.variables['lon'][:]
    
    # Subtract 40,055 to give Oct 1 the same 'date' value in both time and time_obs
    time_obs = time_obs_raw-40055

    # Determine the index in the requested forecast for the desired day of the season 
    id_DayOfTheSeason = np.abs(time-dayOfTheSeason).argmin()

    # Finds the index of the value (in the observation dataset) that is equal to the first element of the hindcast
    id_time_begin = np.abs(time_obs-time[0]).argmin()
    id_time_end   = np.abs(time_obs-time[-1]).argmin()

    # The plus one ensures that the final value is included
    time_obs_refined = time_obs[id_time_begin:id_time_end+1] 
    
    # Determine the indexes for region of interest
    idn = np.abs(lat-latn).argmin() # north
    ids = np.abs(lat-lats).argmin() # south
    
    # Align the longitude data depending on format
    if max(lon) > 190: # if data ranges from 0 - 360
        idw = np.abs(lon-(360+lonw)).argmin()
        ide = np.abs(lon-(360+lone)).argmin()
    else:              # if data ranges from -180 - 180
        idw = np.abs(lon-lonw).argmin()
        ide = np.abs(lon-lone).argmin()

    # Determine indexes for region of interest
    idn_obs = np.abs(lat_obs-latn).argmin()
    ids_obs = np.abs(lat_obs-lats).argmin()
    idw_obs = np.abs(lon_obs-lonw).argmin()
    ide_obs = np.abs(lon_obs-lone).argmin()

    # Obtain lat and lon values within the region of interest
    lat_plt     = lat[ids:idn] 
    lat_plt_obs = lat_obs[ids_obs:idn_obs]
    lon_plt     = lon[idw:ide]
    lon_plt_obs = lon_obs[idw_obs:ide_obs]
    
    # Select the precipitation data that aligns the observation data with the hindcast
    # data on a temporal scale
    precip_obs_refined = rootgrp_obs.variables['pr'][id_time_begin:id_time_end+1][:]
    
    # Select the precipitation data in the user specified region of interest
    precip_aligned     = precipDayXX[:,ids:idn,idw:ide]
    precip_aligned_obs = precip_obs_refined[:,ids_obs:idn_obs,idw_obs:ide_obs]
    
    return precip_aligned, precip_aligned_obs, lat_plt, lat_plt_obs, id_DayOfTheSeason